## TTC Subway Delay Analysis Project

In [12]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
# ttc_codes = pd.read_csv('CSV Files\Code Descriptions.csv')
# ttc_codes.head()

ttc_subway_delay_24 = pd.read_csv('CSV Files/ttc-subway-delay-data-2024.csv')
# ttc_subway_delay_24.head(10)

# Updated as of Nov 24th, 2025
ttc_subway_delay_25 = pd.read_csv('CSV Files/TTC Subway Delay Data since 2025.csv')
# ttc_subway_delay_25.head(10)

ttc_codes = pd.read_json('CSV Files/Code Descriptions.json')
ttc_codes.rename(columns= {"CODE":"Code"}, inplace=True)
# ttc_codes

In [14]:
# Merged DSet
ttc_all_delays = pd.concat([ttc_subway_delay_24, ttc_subway_delay_25], ignore_index=True)
ttc_all_delays.head(10)

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,_id
0,2024-01-01,02:00,Monday,SHEPPARD STATION,MUI,0,0,N,YU,5491,NaN
1,2024-01-01,02:00,Monday,DUNDAS STATION,MUIS,0,0,N,YU,0,NaN
2,2024-01-01,02:08,Monday,DUNDAS STATION,MUPAA,4,10,N,YU,6051,NaN
3,2024-01-01,02:13,Monday,KENNEDY BD STATION,PUTDN,10,16,E,BD,5284,NaN
4,2024-01-01,02:22,Monday,BLOOR STATION,MUPAA,4,10,N,YU,5986,NaN
5,2024-01-01,02:25,Monday,ST CLAIR STATION,MUPAA,3,9,N,YU,6051,NaN
6,2024-01-01,02:25,Monday,BLOOR STATION,MUIRS,0,0,S,YU,0,NaN
7,2024-01-01,02:27,Monday,WOODBINE STATION,EUDO,7,13,E,BD,5077,NaN
8,2024-01-01,02:28,Monday,FINCH STATION,MUIRS,0,0,S,YU,5561,NaN
9,2024-01-01,02:30,Monday,DAVISVILLE STATION,MUI,13,19,N,YU,6051,NaN


Separating Datasets by Line Sections, and Directions (Bound)

### Cuong's Section

### Megans Section

#### Looking at the Merged Data Further

We see that there are 664 unique "station" names. This means there are anomolies/deviations from the traditional station naming process. Once such example is: St.George Station to College Station. From this we learn they aggregate multi-station delays into the [beginning] to [ending] station.

In [ ]:
# Station Masterlist, In Order

# Line 2
LINE_2 = [
    "KIPLING","ISLINGTON", "ROYAL YORK", "OLD MILL", "JANE", "RUNNYMEDE", "HIGH PARK", "KEELE", "DUFFERIN","OSSINGTON", "CHRISTIE",
    "BATHURST","SPADINA","ST GEORGE","BAY","YONGE","SHERBOURNE","CASTLE FRANK","BROADVIEW",
    "CHESTER","PAPE", "DONLANDS","GREENWOOD","COXWELL","WOODBINE","MAIN STREET","VICTORIA PARK","WARDEN","KENNEDY BD"
]
# Line 1
LINE_1 = ["FINCH","NORTH YORK CENTRE","SHEPPARD YONGE","YORK MILLS","LAWRENCE","EGLINTON","DAVISVILLE","ST CLAIR","SUMMERHILL","ROSEDALE",
    "BLOOR YONGE","WELLESLEY","COLLEGE","DUNDAS","QUEEN","KING","UNION","ST ANDREW","OSGOODE","ST PATRICK",
    "QUEENS PARK","MUSEUM","ST GEORGE","SPADINA","DUPONT","ST CLAIR WEST","EGLINTON WEST","GLENCAIRN","LAWRENCE WEST","YORKDALE",
    "WILSON","SHEPPARD WEST","DOWNSVIEW PARK","FINCH WEST","YORK UNIVERSITY","PIONEER VILLAGE","HIGHWAY 407","VAUGHAN MC"
]
# Note, sometimes BLOOR YONGE is logged as YO
# ST A or ST An is St Andrew

In [85]:
# pd.set_option('display.max_rows', None)
print(len(ttc_all_delays['Station'].unique())) # 664 Unique Station Delay Row Names...

664


In [ ]:
range_stations = ttc_all_delays[
    # Exceptions for spaces and other things
    ttc_all_delays['Station'].str.contains(r"\s+to\s+", case=False, na=False)
].copy()

# NVM keeping it in, its a hassle

# # Removing the word STATION for readability
# range_stations['Station'] = (
#     range_stations['Station']
#         .str.replace(r"\bSTAT(ION)?\b|\bSTN\b|\bSTA\b|\bST\b", "", regex = True)
#         .str.strip()
# )

# Only Yonge University Line, merging in code descriptions
# Only 217 Instances from 2024-2025
range_stations_YU = range_stations[range_stations["Line"] == "YU"].copy()
range_stations_YU = range_stations_YU.merge(ttc_codes, how = "left", on="Code")
range_stations_YU

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,_id_x,_id_y,DESCRIPTION
0,2024-01-10,13:30,Wednesday,DAVISVILLE TO EGLINTON,MUO,0,0,N,YU,0,NaN,51.0,MISCELLANEOUS OTHER
1,2024-01-15,23:00,Monday,EGLINTON TO YORK MILLS,MUO,0,0,NaN,YU,0,NaN,51.0,MISCELLANEOUS OTHER
2,2024-01-16,23:00,Tuesday,EGLINTON STATION TO YO,MUO,0,0,NaN,YU,0,NaN,51.0,MISCELLANEOUS OTHER
3,2024-01-16,23:20,Tuesday,DAVISVILLE TO EGLINTON,PUTO,13,20,NaN,YU,0,NaN,NaN,NaN
4,2024-01-17,23:01,Wednesday,EGLINTON TO YORK MILLS,MUO,0,0,NaN,YU,0,NaN,51.0,MISCELLANEOUS OTHER
5,2024-01-18,23:00,Thursday,EGLINTON STATION TO YO,MUO,0,0,NaN,YU,0,NaN,51.0,MISCELLANEOUS OTHER
6,2024-01-19,16:55,Friday,DUNDAS STATION TO EGLI,PUTWZ,6,9,N,YU,0,NaN,109.0,WORK ZONES PROBLEMS â TRACK
7,2024-01-31,06:02,Wednesday,UNION TO KING STATIONS,PUSAC,5,0,N,YU,5456,NaN,77.0,SIGNALS AXLE COUNTER BLOCK FAILURE
8,2024-01-31,23:00,Wednesday,SPADINA TO KING,MUO,0,0,NaN,YU,0,NaN,51.0,MISCELLANEOUS OTHER
9,2024-02-01,19:29,Thursday,UNION STATION TO ST AN,SUO,0,0,N,YU,5491,NaN,117.0,SECURITY OTHER


In [ ]:
summary_YU_Delays = (
    range_stations_YU.groupby("DESCRIPTION", as_index = False)
    .agg(
        total_delay_min = ("Min Delay", "sum"),
        avg_delay_min   = ("Min Delay", "mean"),
        max_delay_min   = ("Min Delay", "max"),
        count  = ("Min Delay", "size")
    )
)
summary_YU_Delays.sort_values('total_delay_min')
# descrip_summary = range_stations_YU.groupby('DESCRIPTION')['Min Delay'].sum().sort_values()
# descrip_summary


,DESCRIPTION,total_delay_min,avg_delay_min,max_delay_min,count
15,NO COLLECTOR AVAILABLE,0,0.000000,0,1
1,CLOSURES - PLANNED,0,0.000000,0,25
27,TRANSIT CONTROL RELATED â TOWER CONTROLLER E...,0,0.000000,0,1
19,PLANT MAINTENANCE â OTHER,0,0.000000,0,1
30,WORK REFUSAL,0,0.000000,0,1
9,INJURED/ILL CUSTOMER IN STATION - TRANSPORTED,0,0.000000,0,1
10,INJURED/ILL CUSTOMER IN STATION â MEDICAL AI...,0,0.000000,0,1
13,MISCELLANEOUS GENERAL SUBWAY LINE DELAYS -,0,0.000000,0,4
2,CONTROLS IMPROPERLY BLOWN DOWN,4,4.000000,4,1
4,DOOR PROBLEMS RE:FAULTY EQUIPMENT,4,4.000000,4,1


In [17]:
# Total Minimum Delay is 2265 minutes
range_stations['Min Delay'].sum()

# Total Minimum Gap is 2365 minutes
range_stations['Min Gap'].sum()

2365

#### Basic Delays Analysis

Minimum Delays - Delay (in minutes) to subway service

Min Gap - Time length (in minutes) between trains (likely caused by the delay)

In [18]:
delays = ttc_all_delays.groupby('Station')
# delays.size().sort_values(ascending=False).head(50).sort_values()

delays_mins = ttc_all_delays.groupby('Station')['Min Delay'].sum().sort_values(ascending = False)
delays_mins

Station
EGLINTON STATION          4594
KIPLING STATION           4145
KENNEDY BD STATION        4012
BLOOR STATION             3737
FINCH STATION             3665
WILSON STATION            3321
DAVISVILLE STATION        3310
SHEPPARD WEST STATION     3050
LAWRENCE STATION          2854
WARDEN STATION            2794
UNION STATION             2788
YORK MILLS STATION        2647
ROSEDALE STATION          2503
YONGE BD STATION          2465
ST CLAIR STATION          2419
ST GEORGE YUS STATION     2373
DUNDAS STATION            2365
YORKDALE STATION          2159
VICTORIA PARK STATION     2131
KEELE STATION             2080
COLLEGE STATION           1965
GREENWOOD STATION         1941
KING STATION              1919
WELLESLEY STATION         1910
DON MILLS STATION         1879
SHEPPARD-YONGE STATION    1843
ST CLAIR WEST STATION     1816
ISLINGTON STATION         1793
SHEPPARD STATION          1785
BROADVIEW STATION         1769
MUSEUM STATION            1754
ST GEORGE BD STATION      1718


In [81]:
delays_gap = ttc_all_delays.groupby('Station')['Min Gap'].sum().sort_values(ascending = False)
delays_gap

Station
KIPLING STATION           6639
KENNEDY BD STATION        6388
EGLINTON STATION          6210
BLOOR STATION             5749
FINCH STATION             5679
WILSON STATION            4580
DAVISVILLE STATION        4486
LAWRENCE STATION          4085
SHEPPARD WEST STATION     3991
WARDEN STATION            3945
ST CLAIR STATION          3698
DUNDAS STATION            3684
ROSEDALE STATION          3552
UNION STATION             3447
YONGE BD STATION          3442
ST GEORGE YUS STATION     3380
YORK MILLS STATION        3195
VICTORIA PARK STATION     3195
COLLEGE STATION           3160
WELLESLEY STATION         2983
YORKDALE STATION          2902
SHEPPARD STATION          2810
DON MILLS STATION         2792
SHEPPARD-YONGE STATION    2791
COXWELL STATION           2782
ST CLAIR WEST STATION     2744
KEELE STATION             2743
ST GEORGE BD STATION      2604
GREENWOOD STATION         2603
BROADVIEW STATION         2568
ISLINGTON STATION         2484
SUMMERHILL STATION        2403
